In [1]:
using Pkg
Pkg.activate("/home/karen/projects/AutonomousRiskFramework/STLCG.jl/.")

 Activating environment at `~/projects/AutonomousRiskFramework/STLCG.jl/Project.toml`


In [2]:
using Revise
using STLCG
using EllipsisNotation


┌ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1278
┌ Info: Precompiling STLCG [6ababdf7-4701-4f28-b3b2-5630f64e3c98]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **



In [3]:
bs = 2
t = 15
x_dim = 3
dim = 2
x = rand(bs, t, x_dim);
xs = [x[:,i:i,..] for i in 1:t];


op = Always(subformula=x, interval=[5, Inf])
h0 = STLCG.init_rnn_cell!(op, x)
scale = 0
distributed = true

# rnn_cell(op, xs[1], op.h0; scale, distributed)
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

true

In [7]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work


(nothing,)

In [8]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

(2×1×3 Fill{Float64}: entries equal to 1.0,)

In [9]:
vcat(STLCG.gradient(x -> sum(STLCG.run_rnn_cell(op, x, x, scale=0)[1][:,end,:]), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [60]:

next_function(formula::STLCG.TemporalFormula) = [formula.subformula]
next_function(formula::Union{STLCG.LessThan, STLCG.GreaterThan, STLCG.Equal}) = [formula.lhs, formula.rhs]
next_function(formula::STLCG.Negation) = [formula.subformula]
next_function(formula::Union{STLCG.Implies, STLCG.And, STLCG.Or, STLCG.Until, STLCG.Then}) = [formula.subformula1, formula.subformula2]


next_function (generic function with 4 methods)

In [5]:
args = Dict(:dims=>4, :pscale=>1, :scale=>0, :keepdims=>true, :distributed=>false)

Dict{Symbol,Integer} with 5 entries:
  :keepdims    => true
  :dims        => 4
  :distributed => false
  :scale       => 0
  :pscale      => 1

In [138]:
not_ϕ = STLCG.Negation(ϕ)

STLCG.Negation(STLCG.LessThan(:x, 5))

In [139]:
robustness_trace(ϕ, x; args...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 4.33878  4.55352  4.0096   4.28465  …  4.87325  4.24155  4.18938  4.36833
 4.35477  4.20984  4.53724  4.8834      4.25997  4.16101  4.62856  4.13871

[:, :, 2] =
 4.19292  4.05989  4.26052  4.3984   …  4.02068  4.74747  4.05824  4.06667
 4.13728  4.53258  4.77843  4.72391     4.18943  4.02867  4.20342  4.66529

[:, :, 3] =
 4.47263  4.89431  4.49562  4.28735  …  4.9606   4.3826   4.59063  4.70413
 4.29403  4.08497  4.01783  4.86847     4.10466  4.29218  4.78882  4.30469

In [140]:
□ϕ = □(ϕ, interval=[2,4])

Always
  subformula: STLCG.LessThan
  interval: Array{Int64}((2,)) [2, 4]
  _interval: Array{Int64}((2,)) [2, 4]
  rnn_dim: Int8 4
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))
  M: Array{Float64}((4, 4)) [0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0]
  b: Array{Float64}((4,)) [0.0, 0.0, 0.0, 1.0]


In [141]:
robustness_trace(□ϕ, x; args...)

LoadError: MethodError: no method matching robustness_trace(::Always, ::Array{Float64,3}; keepdims=true, dims=4, distributed=false, scale=0, pscale=1)
Closest candidates are:
  robustness_trace(!Matched::STLCG.Or, ::Any; dims, pscale, scale, keepdims, distributed) at In[54]:44
  robustness_trace(!Matched::STLCG.LessThan, ::Any; pscale, kwargs...) at In[54]:2
  robustness_trace(!Matched::STLCG.Implies, ::Any; pscale, scale, keepdims, distributed) at In[54]:7 got unsupported keyword argument "dims"
  ...

In [ ]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

In [4]:
ϕ = STLCG.LessThan(:x, 5)
ψ = STLCG.GreaterThan(:x, -3)
op = STLCG.Or(subformula1=STLCG.And(subformula1=ϕ, subformula2=ψ), subformula2=ϕ)

STLCG.Or
  subformula1: STLCG.And
  subformula2: STLCG.LessThan
  operation: Maxish (function of type typeof(Maxish))


In [191]:
(f::STLCG.Formula)(x::Array{Float64}; kwargs...) = robustness_trace(f, x; kwargs...)

In [192]:
ϕ(x)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 4.33878  4.55352  4.0096   4.28465  …  4.87325  4.24155  4.18938  4.36833
 4.35477  4.20984  4.53724  4.8834      4.25997  4.16101  4.62856  4.13871

[:, :, 2] =
 4.19292  4.05989  4.26052  4.3984   …  4.02068  4.74747  4.05824  4.06667
 4.13728  4.53258  4.77843  4.72391     4.18943  4.02867  4.20342  4.66529

[:, :, 3] =
 4.47263  4.89431  4.49562  4.28735  …  4.9606   4.3826   4.59063  4.70413
 4.29403  4.08497  4.01783  4.86847     4.10466  4.29218  4.78882  4.30469

In [147]:
struct Predicate
    x::Symbol
end

In [157]:
(p::Primitive)(x::Int) = (2*x, p.x)
(p::Primitive)(x::Real) = 3*x

In [158]:
a = Primitive(:asadsadsa)

Primitive(:asadsadsa)

In [159]:
a(1), a(1.)

((2, :asadsadsa), 3.0)

In [148]:
Primitive(:x) < 5

Primitive(:x)

In [172]:
Primitive(:a) > Primitive(:b)

false

In [170]:
(Base.:(<))(a::Primitive, b::Primitive) = a.x < b.x